In [28]:
import requests

# Send a GET request to the desired API URL
response = requests.get('https://api.hearthstonejson.com/v1/190920/frFR/cards.collectible.json')

# Parse the response and print it
data = response.json()
#print(data)
type(data)

list

In [18]:
def findLegendaire(data) :
    compteur =0
    for card in data :
        if card["rarity"] == 'LEGENDARY':
            compteur+= 1
    return compteur

def findEpic(data) :
    compteur =0
    for card in data :
        if card["rarity"] == 'EPIC':
            compteur+= 1
    return compteur

def findRare(data) :
    compteur =0
    for card in data :
        if card["rarity"] == 'RARE':      
            compteur+= 1
    return compteur


def findCommon(data) :
    compteur =0
    for card in data :
        if card["rarity"] == 'COMMON':
            compteur+= 1
    return compteur

def findFree(data) :
    compteur =0
    for card in data :
        if card["rarity"] == 'FREE':
            compteur+= 1
    return compteur



def dataSetInfo(data):
    long = len(data)
    print("il y a ",long," cartes au total")
    print(findLegendaire(data)," légendaire sois ",(findLegendaire(data)/long)*100, "% du dataset")
    print(findEpic(data)," epic sois ",(findEpic(data)/long)*100, "% du dataset")
    print(findRare(data)," rare sois ",(findRare(data)/long)*100, "% du dataset")
    print(findCommon(data)," commune sois ",(findCommon(data)/long)*100, "% du dataset")
    print(findFree(data)," free sois ",(findFree(data)/long)*100, "% du dataset")

def find(data, attibut) :
    compteur = {}
    for card in data :
        valeur=card.get(attibut)
        if valeur in compteur:
            compteur[valeur]+=1
        else:
            compteur[valeur]=1
    return( compteur)
    

dataSetInfo(data)

il y a  5845  cartes au total
855  légendaire sois  14.627887082976903 % du dataset
887  epic sois  15.175363558597091 % du dataset
1446  rare sois  24.739093242087254 % du dataset
2029  commune sois  34.713430282292556 % du dataset
628  free sois  10.744225834046194 % du dataset


In [14]:
import csv

def save_csv(dictio, attributs, name):
    with open(name, mode='w', newline='', encoding='utf-8') as fichier_csv:
        writer = csv.DictWriter(fichier_csv, fieldnames=attributs)
        writer.writeheader()
        for objet in dictio:
            ligne = {attr: objet.get(attr, '') for attr in attributs}
            writer.writerow(ligne)


save_csv(data,["id","name","artist","cardClass","cost","attack","health","set","type","rarity"], "dataMK1.csv")
save_csv(data,["faction"], "data_test_trash.csv")

# retiré faction car 5600 et quelque variable était NaN
# retiré collectible, que des true

In [4]:
import pandas as pd
import seaborn as sns
import numpy as np

In [15]:
hs = pd.read_csv("dataMK1.csv")
hs.head()

,id,name,artist,cardClass,cost,attack,health,set,type,rarity
0,AT_001,Lance de flammes,Nutthapon Petchthai,MAGE,5.0,NaN,NaN,TGT,SPELL,COMMON
1,AT_002,Effigie,Tooth,MAGE,3.0,NaN,NaN,TGT,SPELL,RARE
2,AT_003,Héros défunt,Arthur Bozonnet,MAGE,2.0,3.0,2.0,TGT,MINION,RARE
3,AT_004,Déflagration des Arcanes,Gabor Szikszai,MAGE,1.0,NaN,NaN,TGT,SPELL,EPIC
4,AT_005,Métamorphose : sanglier,Mike Sass,MAGE,3.0,NaN,NaN,TGT,SPELL,RARE


In [42]:
val=["attack","health"]

def comptNaN(fichier, col):
    colselect=hs[col]
    compteur=colselect.isna().sum()
    return compteur

#comptNaN(hs, val)

df = pd.DataFrame(data) 
df["mechanics"] = df["mechanics"].fillna(0)
#df["mechanics"] = df["mechanics"].apply(lambda x: [str(val) for val in x] if  isinstance(x, list) else x)
print(df["mechanics"])

flomec=df["mechanics"].apply(lambda x: isinstance(x,float))
print(flomec)

0                                         0
1                                  [SECRET]
2                        [HEROPOWER_DAMAGE]
3       [RECEIVES_DOUBLE_SPELLDAMAGE_BONUS]
4                                         0
                       ...                 
5840                       [RUSH, WINDFURY]
5841                            [BATTLECRY]
5842                                      0
5843                       [TRIGGER_VISUAL]
5844                       [TRIGGER_VISUAL]
Name: mechanics, Length: 5845, dtype: object
0       False
1       False
2       False
3       False
4       False
        ...  
5840    False
5841    False
5842    False
5843    False
5844    False
Name: mechanics, Length: 5845, dtype: bool


In [50]:
def mechIneEx(nomExc):
    df = pd.DataFrame(data) 

    df["mechanics"] = df["mechanics"].fillna("miss")
    
    for valeur in df["mechanics"].explode().unique():
        df[f"is_{valeur}"] = df["mechanics"].apply(lambda x:1 if valeur in x else 0)
    
    df.drop(columns=["mechanics"], inplace = True)
    df.to_csv(nomExc, index=False)

mechIneEx("data_mech.csv")

mech = pd.read_csv("data_mech.csv")
mech.tail()
mech.shape
mech.tail()

,artist,cardClass,collectible,cost,dbfId,flavor,id,name,rarity,set,...,is_MANATHIRST,is_DREDGE,is_COLOSSAL,is_SPELLBURST,is_OVERKILL,is_FORGE,is_TITAN,is_QUEST,is_EXCAVATE,is_QUICKDRAW
5840,Matt Dixon,NEUTRAL,True,2.0,61969,Un vrai pince-sans-rire…,YOP_031,Chevaucheur de crabe,COMMON,DARKMOON_FAIRE,...,0,0,0,0,0,0,0,0,0,0
5841,Maria Trepalina,NEUTRAL,True,1.0,61970,Vous me semblez du genre à acheter trois armur...,YOP_032,Vendeuse d’armures,RARE,DARKMOON_FAIRE,...,0,0,0,0,0,0,0,0,0,0
5842,Anton Zemskov,WARLOCK,True,3.0,61971,C’est la dernière fois que je laisse un diablo...,YOP_033,Retour de flammes,COMMON,DARKMOON_FAIRE,...,0,0,0,0,0,0,0,0,0,0
5843,Andrew Hou,NEUTRAL,True,9.0,61972,Ça ne fait même pas 5 minutes que nous avons e...,YOP_034,Aile noire en fuite,RARE,DARKMOON_FAIRE,...,0,0,0,0,0,0,0,0,0,0
5844,Zoltan Boros,NEUTRAL,True,5.0,61973,Tête-de-Lune ne sonnait pas aussi bien.,YOP_035,Croc-de-Lune,LEGENDARY,DARKMOON_FAIRE,...,0,0,0,0,0,0,0,0,0,0


In [59]:
delete = ['artist', 'cardClass', 'collectible', 'cost', 'dbfId', 'flavor', 'id', 'name', 'rarity', 'set', 'spellSchool', 'text', 'type', 'attack', 'health', 'race', 'races', 'referencedTags', 'elite', 'targetingArrowText', 'durability', 'overload', 'spellDamage', 'battlegroundsPremiumDbfId', 'techLevel', 'hasDiamondSkin', 'howToEarnGolden', 'armor', 'heroPowerDbfId', 'howToEarn', 'collectionText', 'faction', 'countAsCopyOfDbfId', 'classes', 'multiClassGroup', 'isMiniSet', 'hideStats', 'questReward', 'is_miss']

#mech.drop(columns=delete,inplace=True)

print(mech.columns.tolist())

['is_SECRET', 'is_HEROPOWER_DAMAGE', 'is_RECEIVES_DOUBLE_SPELLDAMAGE_BONUS', 'is_INSPIRE', 'is_BATTLECRY', 'is_DEATHRATTLE', 'is_OVERHEAL', 'is_TRIGGER_VISUAL', 'is_InvisibleDeathrattle', 'is_COMBO', 'is_CHOOSE_ONE', 'is_AURA', 'is_OVERLOAD', 'is_CHARGE', 'is_DIVINE_SHIELD', 'is_FORGETFUL', 'is_SPELLPOWER', 'is_STEALTH', 'is_TAUNT', 'is_CANT_ATTACK', 'is_TOPDECK', 'is_DISCOVER', 'is_HONORABLEKILL', 'is_RUSH', 'is_WINDFURY', 'is_ImmuneToSpellpower', 'is_FRENZY', 'is_POISONOUS', 'is_LIFESTEAL', 'is_OUTCAST', 'is_MAGNETIC', 'is_AFFECTED_BY_SPELL_POWER', 'is_CANT_BE_TARGETED_BY_SPELLS', 'is_CANT_BE_TARGETED_BY_HERO_POWERS', 'is_GEARS', 'is_FINISH_ATTACK_SPELL_ON_DAMAGE', 'is_ECHO', 'is_FREEZE', 'is_GRIMY_GOONS', 'is_KABAL', 'is_JADE_GOLEM', 'is_JADE_LOTUS', 'is_TRADEABLE', 'is_ADJACENT_BUFF', 'is_ENRAGED', 'is_DEATH_KNIGHT', 'is_REBORN', 'is_START_OF_GAME', 'is_TWINSPELL', 'is_MULTIPLY_BUFF_VALUE', 'is_CORRUPT', 'is_SIDEQUEST', 'is_SILENCE', 'is_COLLECTIONMANAGER_FILTER_MANA_EVEN', 'is_COL

In [60]:
#mech.head()
mech.shape

(5845, 66)

In [65]:
lien = pd.concat([hs,mech], axis=1)

lien.to_csv("dataMKII.csv", index=False)

In [68]:
MK2= pd.read_csv("dataMKII.csv")

MK2.tail()
#MK2.shape


,id,name,artist,cardClass,cost,attack,health,set,type,rarity,...,is_MANATHIRST,is_DREDGE,is_COLOSSAL,is_SPELLBURST,is_OVERKILL,is_FORGE,is_TITAN,is_QUEST,is_EXCAVATE,is_QUICKDRAW
5840,YOP_031,Chevaucheur de crabe,Matt Dixon,NEUTRAL,2.0,1.0,4.0,DARKMOON_FAIRE,MINION,COMMON,...,0,0,0,0,0,0,0,0,0,0
5841,YOP_032,Vendeuse d’armures,Maria Trepalina,NEUTRAL,1.0,1.0,3.0,DARKMOON_FAIRE,MINION,RARE,...,0,0,0,0,0,0,0,0,0,0
5842,YOP_033,Retour de flammes,Anton Zemskov,WARLOCK,3.0,NaN,NaN,DARKMOON_FAIRE,SPELL,COMMON,...,0,0,0,0,0,0,0,0,0,0
5843,YOP_034,Aile noire en fuite,Andrew Hou,NEUTRAL,9.0,9.0,9.0,DARKMOON_FAIRE,MINION,RARE,...,0,0,0,0,0,0,0,0,0,0
5844,YOP_035,Croc-de-Lune,Zoltan Boros,NEUTRAL,5.0,6.0,3.0,DARKMOON_FAIRE,MINION,LEGENDARY,...,0,0,0,0,0,0,0,0,0,0
